# BentoML Example: Fast AI with Tabular data

This notebook is based on fastai's cours v3 lesson 4.  We are going to train a model that predict salary range base on the data we provided.

[BentoML](http://bentoml.ai) is an open source platform for machine learning model serving and deployment. In this project we will use BentoML to package the trained fast.ai model, and build a containerized REST API model server.


![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=fast-ai&ea=fast-ai-salary-range-prediction&dt=fast-ai-salary-range-prediction)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install fastai
!pip install bentoml

In [2]:
from fastai.tabular import *

## Prepare Training Data

In [3]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [4]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']
procs = [FillMissing, Categorify, Normalize]

In [5]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

In [6]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [7]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,False,-0.4095,3.0509,-0.4224,<50k
Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,0.5434,-1.1076,-0.0312,<50k
Self-emp-not-inc,Bachelors,Never-married,Sales,Not-in-family,White,False,0.4701,1.2268,1.1422,>=50k
Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,False,0.3235,0.0173,-0.4224,<50k
Private,HS-grad,Never-married,#na#,Not-in-family,White,True,-0.8493,-0.3763,-0.0312,<50k
Private,HS-grad,Never-married,Machine-op-inspct,Unmarried,Black,False,-0.8493,-0.4789,-0.4224,<50k
Private,Bachelors,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,False,-0.6294,1.0659,1.1422,<50k
Private,12th,Never-married,Other-service,Own-child,White,False,-1.5090,-0.5379,-0.8135,<50k
Self-emp-inc,10th,Never-married,Sales,Own-child,White,False,-1.5823,-0.0769,-1.5958,<50k
?,Some-college,Never-married,?,Own-child,Black,False,-1.4357,-0.1468,-0.0312,<50k


## Model Training

In [8]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)

In [9]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.365644,0.370601,0.840000,00:03


In [10]:
row = df.iloc[0] # sample input date for testing

learn.predict(row)

(Category >=50k, tensor(1), tensor([0.4036, 0.5964]))

## Create BentoService for model serving

In [11]:
%%writefile tabular_csv.py

from bentoml import env, api, artifacts, BentoService
from bentoml.artifact import FastaiModelArtifact
from bentoml.handlers import DataframeHandler


@env(pip_dependencies=['fastai'])
@artifacts([FastaiModelArtifact('model')])
class FastaiTabularModel(BentoService):
    
    @api(DataframeHandler)
    def predict(self, df):
        results = []
        for _, row in df.iterrows():       
            prediction = self.artifacts.model.predict(row)
            results.append(prediction[0].obj)
        return results

Overwriting tabular_csv.py


## Save BentoService to file archive

In [12]:
# 1) import the custom BentoService defined above
from tabular_csv import FastaiTabularModel

# 2) `pack` it with required artifacts
svc = FastaiTabularModel()
svc.pack('model', learn)

# 3) save your BentoSerivce
saved_path = svc.save()

[2020-02-14 12:57:51,895] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 12:57:52,324] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml

no previously-included directories found matching 'examples'
no previously-included directories found matching 'tests'
no previously-included directories found matching 'docs'
no previously-included directories found matching 'scripts'


writing manifest file 'BentoML.egg-info/SOURCES.txt'
running check


creating BentoML-0.6.2+8.gd95a887
creating BentoML-0.6.2+8.gd95a887/BentoML.egg-info
creating BentoML-0.6.2+8.gd95a887/bentoml
creating BentoML-0.6.2+8.gd95a887/bentoml/artifact
creating BentoML-0.6.2+8.gd95a887/bentoml/bundler
creating BentoML-0.6.2+8.gd95a887/bentoml/cli
creating BentoML-0.6.2+8.gd95a887/bentoml/clipper
creating BentoML-0.6.2+8.gd95a887/bentoml/configuration
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment/aws_lambda
creating BentoML-0.6.2+8.gd95a887/bentoml/deployment/sagemaker
creating BentoML-0.6.2+8.gd95a887/bentoml/handlers
creating BentoML-0.6.2+8.gd95a887/bentoml/marshal
creating BentoML-0.6.2+8.gd95a887/bentoml/migrations
creating BentoML-0.6.2+8.gd95a887/bentoml/migrations/versions
creating BentoML-0.6.2+8.gd95a887/bentoml/proto
creating BentoML-0.6.2+8.gd95a887/bentoml/repository
creating BentoML-0.6.2+8.gd95a887/bentoml/server
creating BentoML-0.6.2+8.gd95a887/bentoml/server/static
creating BentoML-0

copying bentoml/proto/yatai_service_pb2.py -> BentoML-0.6.2+8.gd95a887/bentoml/proto
copying bentoml/proto/yatai_service_pb2_grpc.py -> BentoML-0.6.2+8.gd95a887/bentoml/proto
copying bentoml/repository/__init__.py -> BentoML-0.6.2+8.gd95a887/bentoml/repository
copying bentoml/repository/metadata_store.py -> BentoML-0.6.2+8.gd95a887/bentoml/repository
copying bentoml/server/__init__.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/bento_api_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/bento_sagemaker_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/gunicorn_config.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/gunicorn_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/marshal_server.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/middlewares.py -> BentoML-0.6.2+8.gd95a887/bentoml/server
copying bentoml/server/utils.py -> BentoML-0.6.2+8.gd95a

## Use BentoService with BentoML CLI

**Use `bentoml get` to list all versions of BentoService, including the version tag will display additional information and metadata**

In [13]:
!bentoml get FastaiTabularModel

BENTO_SERVICE                             AGE            APIS                       ARTIFACTS
FastaiTabularModel:20200214125752_4055F5  36.61 seconds  predict<DataframeHandler>  model<FastaiModelArtifact>


In [14]:
!bentoml get FastaiTabularModel:20200214125752_4055F5

{
  "name": "FastaiTabularModel",
  "version": "20200214125752_4055F5",
  "uri": {
    "type": "LOCAL",
    "uri": "/Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200214125752_4055F5"
  },
  "bentoServiceMetadata": {
    "name": "FastaiTabularModel",
    "version": "20200214125752_4055F5",
    "createdAt": "2020-02-14T20:58:16.897556Z",
    "env": {
      "condaEnv": "name: bentoml-FastaiTabularModel\nchannels:\n- defaults\ndependencies:\n- python=3.7.3\n- pip\n",
      "pipDependencies": "bentoml==0.6.2\nfastai",
      "pythonVersion": "3.7.3"
    },
    "artifacts": [
      {
        "name": "model",
        "artifactType": "FastaiModelArtifact"
      }
    ],
    "apis": [
      {
        "name": "predict",
        "handlerType": "DataframeHandler",
        "docs": "BentoService API",
        "handlerConfig": {
          "orient": "records",
          "typ": "frame",
          "input_dtypes": null,
          "output_orient": "records"
        }
      }

  }
}


Quickly get prediction result with `bentoml run`

In [15]:
!bentoml run FastaiTabularModel:20200214125752_4055F5 predict \
    --input https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv

[2020-02-14 12:59:25,233] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 12:59:25,245] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 12:59:25,260] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

Start a local realtime prediction service with `bentoml serve`

## Model Serving via REST API

*Note: Running as local rest api server does not work with Google Colab, please copy this notebook to run it locally*

In [ ]:
!bentoml serve {saved_path}

 * Serving Flask app "TabularModel" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2019 15:08:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [24/Oct/2019 15:08:27] "POST /predict HTTP/1.1" 200 -


### Send prediction requeset to the REST API server

#### JSON Request

```bash
curl -X POST \
  http://localhost:5000/predict \
  -H 'Content-Type: application/json' \
  -d '[{
  "age": 49,
  "workclass": "Private",
  "fnlwgt": 101320,
  "education": "Assoc-acdm",
  "education-num": 12.0,
  "marital-status": "Married-civ-spouse",
  "occupation": "",
  "relationship": "Wift",
  "race": "White",
  "sex": "Female",
  "capital-gain": 0,
  "capital-loss": 1902,
  "hours-per-week": 40,
  "native-country": "United-States",
  "salary": ">=50k"
}]'
```

#### CSV Request

```bash
curl -X POST "http://127.0.0.1:5000/predict" \
    -H "Content-Type: text/csv" \
    --data-binary @test.csv
```

## Install saved BentoService as PyPI package

In [16]:
!pip install {saved_path}

Processing /Users/bozhaoyu/bentoml/repository/FastaiTabularModel/20200214125752_4055F5


  Created wheel for FastaiTabularModel: filename=FastaiTabularModel-20200214125752_4055F5-py3-none-any.whl size=253287 sha256=eefa4b0882d2f3cbffdc36823fe5cc70ccd31a9b60df07808b3ec59fd683d513
  Stored in directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/pip-ephem-wheel-cache-c5vwp5hd/wheels/0a/f1/25/06c81c2881a541c58f44c3e1f0d968c9a2bfbf281323a651d4
Successfully built FastaiTabularModel


In [17]:
!FastaiTabularModel info

[2020-02-14 13:01:34,234] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 13:01:34,248] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
{
  "name": "FastaiTabularModel",
  "version": "20200214125752_4055F5",
  "created_at": "2020-02-14T20:58:16.897556Z",
  "env": {
    "conda_env": "name: bentoml-FastaiTabularModel\nchannels:\n- defaults\ndependencies:\n- python

In [18]:
# Use CSV data
!FastaiTabularModel run predict \
--input=https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.csv

[2020-02-14 13:01:48,246] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 13:01:48,255] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 13:01:48,257] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

In [20]:
# Use json data
!FastaiTabularModel run predict \
--input=https://raw.githubusercontent.com/bentoml/gallery/master/fast-ai/salary-range-prediction/test.json

[2020-02-14 13:02:03,680] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be default to same version as last PyPI release at version: 0.6.2. You can also force bentoml to use a specific version for deploying your BentoService bundle, by setting the config 'core/bentoml_deploy_version' to a pinned version or your custom BentoML on github, e.g.:'bentoml_deploy_version = git+https://github.com/{username}/bentoml.git@{branch}'
[2020-02-14 13:02:03,689] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 13:02:03,691] WARNING - BentoML local changes detected - Local BentoML repository including all code changes will be bundled together with the BentoService bundle. When used with docker, the base docker image will be

### Use BentoService with Docker

Use the auto generated `Dockerfile` inside the BentoService file bundle to build docker image

In [36]:
!cd {saved_path} && docker build -t fastai-salary .

Sending build context to Docker daemon  1.181MB
Step 1/12 : FROM continuumio/miniconda3:4.7.12
 ---> 406f2b43ea59
Step 2/12 : ENTRYPOINT [ "/bin/bash", "-c" ]
 ---> Running in feff1870c1ef
Removing intermediate container feff1870c1ef
 ---> 5554f67fe14d
Step 3/12 : EXPOSE 5000
 ---> Running in fda6256732f5
Removing intermediate container fda6256732f5
 ---> 138d488f26e6
Step 4/12 : RUN set -x      && apt-get update      && apt-get install --no-install-recommends --no-install-suggests -y libpq-dev build-essential      && rm -rf /var/lib/apt/lists/*
 ---> Running in 5a66ee5072a7
+ apt-get update
Get:1 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:2 http://deb.debian.org/debian buster InRelease [122 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:4 http://security.debian.org/debian-security buster/updates/main amd64 Packages [176 kB]
Get:5 http://deb.debian.org/debian buster/main amd64 Packages [7907 kB]
Fetched 8319 kB in 3s

Selecting previously unselected package libatomic1:amd64.
Preparing to unpack .../15-libatomic1_8.3.0-6_amd64.deb ...
Unpacking libatomic1:amd64 (8.3.0-6) ...
Selecting previously unselected package libasan5:amd64.
Preparing to unpack .../16-libasan5_8.3.0-6_amd64.deb ...
Unpacking libasan5:amd64 (8.3.0-6) ...
Selecting previously unselected package liblsan0:amd64.
Preparing to unpack .../17-liblsan0_8.3.0-6_amd64.deb ...
Unpacking liblsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libtsan0:amd64.
Preparing to unpack .../18-libtsan0_8.3.0-6_amd64.deb ...
Unpacking libtsan0:amd64 (8.3.0-6) ...
Selecting previously unselected package libubsan1:amd64.
Preparing to unpack .../19-libubsan1_8.3.0-6_amd64.deb ...
Unpacking libubsan1:amd64 (8.3.0-6) ...
Selecting previously unselected package libmpx2:amd64.
Preparing to unpack .../20-libmpx2_8.3.0-6_amd64.deb ...
Unpacking libmpx2:amd64 (8.3.0-6) ...
Selecting previously unselected package libquadmath0:amd64.
Preparing to unp

mkl_random-1.1.0     | 321 KB    | ########## | 100% 
blas-1.0             | 6 KB      | ########## | 100% 
mkl-service-2.3.0    | 218 KB    | ########## | 100% 
conda-4.8.2          | 2.8 MB    | ########## | 100% 
libgfortran-ng-7.3.0 | 1006 KB   | ########## | 100% 
numpy-base-1.18.1    | 4.2 MB    | ########## | 100% 
openssl-1.1.1d       | 2.5 MB    | ########## | 100% 
numpy-1.18.1         | 5 KB      | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Removing intermediate container ae780e64a9ef
 ---> 71c4b2fac222
Step 6/12 : COPY . /bento
 ---> 88729c86f348
Step 7/12 : WORKDIR /bento
 ---> Running in 459ae51da3b7
Removing intermediate container 459ae51da3b7
 ---> d2d8d2f73303
Step 8/12 : RUN if [ -f /bento/setup.sh ]; then /bin/bash -c /bento/setup.sh; fi
 ---> Running in 23bff85f7363
Removing intermediate container 23bff85f7363
 ---> 2d1bc34185c3
Step 9/12 : RUN conda env update -n b

  Created wheel for tabulate: filename=tabulate-0.8.6-py3-none-any.whl size=23273 sha256=a699875683ae32cb6bb5094d30e4b8fa3cc7df366a20309340bfa76170237d39
  Stored in directory: /root/.cache/pip/wheels/09/b6/7e/08b4ee715a1239453e89a59081f0ac369a9036f232e013ecd8
  Created wheel for python-json-logger: filename=python_json_logger-0.1.11-py2.py3-none-any.whl size=5076 sha256=d09d98a0080952f6fae9a95e8ce53470205e1cd14b100859617cf40faeb51762
  Stored in directory: /root/.cache/pip/wheels/fa/7f/fd/92ccdbb9d1a65486406e0363d2ba5b4ce52f400a915f602ecb
  Created wheel for prometheus-client: filename=prometheus_client-0.7.1-py3-none-any.whl size=41402 sha256=21ab1ef311301f443503682364c128b48e8cc71eac3006b9c5e7b0803c123cc6
  Stored in directory: /root/.cache/pip/wheels/30/0c/26/59ba285bf65dc79d195e9b25e2ddde4c61070422729b0cd914
  Created wheel for alembic: filename=alembic-1.4.0-py2.py3-none-any.whl size=157563 sha256=c2458c589975c0cacb2b9e3800bb36f8f28c3985dc7d6097bdbd05c3bf3bd863
  Stored in direct

  Created wheel for BentoML: filename=BentoML-0.6.2+8.gd95a887-py3-none-any.whl size=513796 sha256=47aab287696ef482c10733fb11d3e1be5819f856756fb5f51e73d12a663cbcac
  Stored in directory: /root/.cache/pip/wheels/63/75/8b/c1e0439890f032f9c89af93138f58408c22c672822c9e60dd1
Successfully built BentoML
  Attempting uninstall: BentoML
    Found existing installation: BentoML 0.6.2
    Uninstalling BentoML-0.6.2:
      Successfully uninstalled BentoML-0.6.2
Removing intermediate container ee60bc4704bb
 ---> a95e372399a1
Step 12/12 : CMD ["bentoml serve-gunicorn /bento"]
 ---> Running in 8f2e9065cf3e
Removing intermediate container 8f2e9065cf3e
 ---> 38fa02890d9e
Successfully built 38fa02890d9e
Successfully tagged fastai-salary:latest


In [37]:
!docker run -p 5000:5000 fastai-salary

[2020-02-14 23:05:15,352] INFO - get_gunicorn_num_of_workers: 3, calculated by cpu count
[2020-02-14 23:05:15 +0000] [1] [INFO] Starting gunicorn 20.0.4
[2020-02-14 23:05:15 +0000] [1] [INFO] Listening at: http://0.0.0.0:5000 (1)
[2020-02-14 23:05:15 +0000] [1] [INFO] Using worker: sync
[2020-02-14 23:05:15 +0000] [9] [INFO] Booting worker with pid: 9
[2020-02-14 23:05:15 +0000] [10] [INFO] Booting worker with pid: 10
[2020-02-14 23:05:15 +0000] [11] [INFO] Booting worker with pid: 11
[2020-02-14 23:05:15,841] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 23:05:15,880] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with BentoML version 0.6.2,  but loading from BentoML version 0.6.2+8.gd95a887
[2020-02-14 23:05:15,931] WARNING - Saved BentoService bundle version mismatch: loading BentoServie bundle create with Bento

# Deploy BentoService as REST API server to the cloud


BentoML support deployment to multiply cloud provider services, such as AWS Lambda, AWS Sagemaker, Google Cloudrun and etc. You can find the full list and guide on the documentation site at https://docs.bentoml.org/en/latest/deployment/index.html

For this project, we are going to deploy to AWS Sagemaker

**`bentoml sagemaker deploy` deploys to AWS Sagemaker with a single command**

In [29]:
!bentoml sagemaker deploy fastai-salary -b FastaiTabularModel:20200214125752_4055F5 --api-name predict --verbose

[2020-02-14 13:35:40,724] DEBUG - Using BentoML with local Yatai server
[2020-02-14 13:35:40,857] DEBUG - Upgrading tables to the latest revision
Deploying Sagemaker deployment \[2020-02-14 13:35:41,631] DEBUG - Created temporary directory: /private/var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/bentoml-temp-50dha7pn
|[2020-02-14 13:35:41,940] DEBUG - Getting docker login info from AWS
[2020-02-14 13:35:41,942] DEBUG - Building docker image: 192023623294.dkr.ecr.us-west-2.amazonaws.com/fastaitabularmodel-sagemaker:20200214125752_4055F5
\[2020-02-14 13:35:42,445] INFO - Step 1/11 : FROM continuumio/miniconda3:4.7.12
[2020-02-14 13:35:42,445] INFO - 

/[2020-02-14 13:36:26,336] INFO -  ---> 406f2b43ea59

[2020-02-14 13:36:26,336] INFO - Step 2/11 : EXPOSE 8080
[2020-02-14 13:36:26,336] INFO - 

|[2020-02-14 13:36:26,434] INFO -  ---> Running in 9cd5c63fd100

/[2020-02-14 13:36:26,660] INFO -  ---> 58636f0540f4

[2020-02-14 13:36:26,660] INFO - Step 3/11 : RUN set -x      && apt-get

\[2020-02-14 13:36:43,313] INFO - Get:32 http://deb.debian.org/debian buster/main amd64 libdpkg-perl all 1.19.7 [1414 kB]

\[2020-02-14 13:36:43,702] INFO - Get:33 http://deb.debian.org/debian buster/main amd64 dpkg-dev all 1.19.7 [1773 kB]

|[2020-02-14 13:36:44,028] INFO - Get:34 http://deb.debian.org/debian buster/main amd64 build-essential amd64 12.6 [7576 B]

\[2020-02-14 13:36:44,172] INFO - debconf: delaying package configuration, since apt-utils is not installed

-[2020-02-14 13:36:44,206] INFO - Fetched 47.2 MB in 11s (4228 kB/s)

[2020-02-14 13:36:44,232] INFO - Selecting previously unselected package binutils-common:amd64.
(Reading database ... 
(Reading database ... 50% INFO - (Reading database ... 5%
(Reading database ... 75% INFO - (Reading database ... 55%
[2020-02-14 13:36:44,237] INFO - (Reading database ... 80%
[2020-02-14 13:36:44,239] INFO - (Reading database ... 85%
[2020-02-14 13:36:44,240] INFO - (Reading database ... 90%
[2020-02-14 13:36:44,241] INFO - (Re

[2020-02-14 13:36:50,239] INFO - Selecting previously unselected package make.
Preparing to unpack .../28-make_4.2.1-1.2_amd64.deb ...

\[2020-02-14 13:36:50,241] INFO - Unpacking make (4.2.1-1.2) ...

[2020-02-14 13:36:50,303] INFO - Selecting previously unselected package libdpkg-perl.

[2020-02-14 13:36:50,305] INFO - Preparing to unpack .../29-libdpkg-perl_1.19.7_all.deb ...

[2020-02-14 13:36:50,308] INFO - Unpacking libdpkg-perl (1.19.7) ...

-[2020-02-14 13:36:50,387] INFO - Selecting previously unselected package dpkg-dev.

[2020-02-14 13:36:50,389] INFO - Preparing to unpack .../30-dpkg-dev_1.19.7_all.deb ...

[2020-02-14 13:36:50,392] INFO - Unpacking dpkg-dev (1.19.7) ...

/[2020-02-14 13:36:50,489] INFO - Selecting previously unselected package build-essential.

[2020-02-14 13:36:50,491] INFO - Preparing to unpack .../31-build-essential_12.6_amd64.deb ...

[2020-02-14 13:36:50,494] INFO - Unpacking build-essential (12.6) ...

[2020-02-14 13:36:50,523] INFO - Selecting pr

[2020-02-14 13:36:53,208] INFO - Get:17 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-dav-ext amd64 1.14.2-2+deb10u1 [100 kB]

[2020-02-14 13:36:53,211] INFO - Get:18 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-echo amd64 1.14.2-2+deb10u1 [104 kB]

[2020-02-14 13:36:53,214] INFO - Get:19 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-geoip amd64 1.14.2-2+deb10u1 [93.8 kB]

[2020-02-14 13:36:53,221] INFO - Get:20 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-image-filter amd64 1.14.2-2+deb10u1 [97.4 kB]

-[2020-02-14 13:36:53,227] INFO - Get:21 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-subs-filter amd64 1.14.2-2+deb10u1 [95.7 kB]

[2020-02-14 13:36:53,244] INFO - Get:22 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-upstream-fair amd64 1.14.2-2+deb10u1 [95.7 kB]

[2020-02-14 13:36:53,253] INFO - Get:23 http://deb.debian.org/debian buster/main amd64 libnginx-mod-http-xsl

[2020-02-14 13:36:55,323] INFO - Selecting previously unselected package libnginx-mod-http-xslt-filter.

[2020-02-14 13:36:55,325] INFO - Preparing to unpack .../22-libnginx-mod-http-xslt-filter_1.14.2-2+deb10u1_amd64.deb ...

[2020-02-14 13:36:55,327] INFO - Unpacking libnginx-mod-http-xslt-filter (1.14.2-2+deb10u1) ...

[2020-02-14 13:36:55,363] INFO - Selecting previously unselected package libnginx-mod-mail.

[2020-02-14 13:36:55,365] INFO - Preparing to unpack .../23-libnginx-mod-mail_1.14.2-2+deb10u1_amd64.deb ...

[2020-02-14 13:36:55,368] INFO - Unpacking libnginx-mod-mail (1.14.2-2+deb10u1) ...

/[2020-02-14 13:36:55,408] INFO - Selecting previously unselected package libnginx-mod-stream.

[2020-02-14 13:36:55,410] INFO - Preparing to unpack .../24-libnginx-mod-stream_1.14.2-2+deb10u1_amd64.deb ...

[2020-02-14 13:36:55,413] INFO - Unpacking libnginx-mod-stream (1.14.2-2+deb10u1) ...

[2020-02-14 13:36:55,452] INFO - Selecting previously unselected package nginx-full.

[2020-

...

-[2020-02-14 13:56:15,214] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfigName': 'bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'c86a8807-cf36-4a32-b6ad-96f89e0f9174', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c86a8807-cf36-4a32-b6ad-96f89e0f9174', 'content-type': 'application/x-amz-json-1.1', 'content-length': '301', 'date': 'Fri, 14 Feb 2020 21:56:14 GMT'}, 'RetryAttempts': 0}}
\[2020-02-14 13:56:20,404] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfi

-[2020-02-14 13:57:12,913] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfigName': 'bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '4a31e836-c42d-44fc-8d9d-377154688366', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '4a31e836-c42d-44fc-8d9d-377154688366', 'content-type': 'application/x-amz-json-1.1', 'content-length': '301', 'date': 'Fri, 14 Feb 2020 21:57:12 GMT'}, 'RetryAttempts': 0}}
|[2020-02-14 13:57:18,110] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfi

-[2020-02-14 13:58:11,476] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfigName': 'bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '323cb048-e120-433d-b6e6-96d29c25a476', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '323cb048-e120-433d-b6e6-96d29c25a476', 'content-type': 'application/x-amz-json-1.1', 'content-length': '301', 'date': 'Fri, 14 Feb 2020 21:58:11 GMT'}, 'RetryAttempts': 0}}
\[2020-02-14 13:58:16,733] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfi

/[2020-02-14 13:59:09,795] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfigName': 'bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '2c59b6b3-94a6-4fa0-b719-b299b24180a4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2c59b6b3-94a6-4fa0-b719-b299b24180a4', 'content-type': 'application/x-amz-json-1.1', 'content-length': '301', 'date': 'Fri, 14 Feb 2020 21:59:09 GMT'}, 'RetryAttempts': 0}}
-[2020-02-14 13:59:15,009] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfi

|[2020-02-14 14:00:08,125] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfigName': 'bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5', 'EndpointStatus': 'Creating', 'CreationTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 2, 14, 13, 51, 39, 158000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '0a4eee2b-7d31-4fdc-95aa-929e8318bc0f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '0a4eee2b-7d31-4fdc-95aa-929e8318bc0f', 'content-type': 'application/x-amz-json-1.1', 'content-length': '301', 'date': 'Fri, 14 Feb 2020 22:00:07 GMT'}, 'RetryAttempts': 0}}
/[2020-02-14 14:00:13,460] DEBUG - AWS describe endpoint response: {'EndpointName': 'bobo-fastai-salary', 'EndpointArn': 'arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary', 'EndpointConfi

List all Sagemaker deployments

In [30]:
!bentoml sagemaker list

NAME           NAMESPACE    PLATFORM       BENTO_SERVICE                             STATUS    AGE
fastai-salary  bobo         aws-sagemaker  FastaiTabularModel:20200214125752_4055F5  running   1 hour and 1 minute


Retrieve the latest information for Sagemaker deployment

In [31]:
!bentoml sagemaker get fastai-salary

{
  "namespace": "bobo",
  "name": "fastai-salary",
  "spec": {
    "bentoName": "FastaiTabularModel",
    "bentoVersion": "20200214125752_4055F5",
    "operator": "AWS_SAGEMAKER",
    "sagemakerOperatorConfig": {
      "region": "us-west-2",
      "instanceType": "ml.m4.xlarge",
      "instanceCount": 1,
      "apiName": "predict"
    }
  },
  "state": {
    "state": "RUNNING",
    "infoJson": {
      "EndpointName": "bobo-fastai-salary",
      "EndpointArn": "arn:aws:sagemaker:us-west-2:192023623294:endpoint/bobo-fastai-salary",
      "EndpointConfigName": "bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5",
      "ProductionVariants": [
        {
          "VariantName": "bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5",
          "DeployedImages": [
            {
              "SpecifiedImage": "192023623294.dkr.ecr.us-west-2.amazonaws.com/fastaitabularmodel-sagemaker:20200214125752_4055F5",
              "ResolvedImage": "192023623294.dkr.ecr.us-west-2.amazon

Test deployed service with `aws` CLI tool

In [34]:
!aws sagemaker-runtime invoke-endpoint \
  --endpoint-name bobo-fastai-salary \
  --body file:///Users/bozhaoyu/src/bento_gallery/fast-ai/salary-range-prediction/test.csv \
  --content-type "text/csv" \
  output.json && cat output.json

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "bobo-fastai-salary-FastaiTabularModel-20200214125752-4055F5"
}
[">=50k"]

`bentoml sagemaker delete` removes the deployment and related resources

In [35]:
!bentoml sagemaker delete fastai-salary

Successfully deleted AWS Sagemaker deployment "fastai-salary"
